# Matching Department of Health data (2011) to M&W

Import Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re #regular expressions 
from fuzzywuzzy import process, fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os 
import glob

###### Read in Data

                                                           M&W Data

In [2]:
pd.set_option('display.max_rows', None)
redFood = pd.read_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Data\Processed\ReducedwithWeights\matchedwithFPS.csv", index_col=0)
redFood.rename(columns={'Similarity Score':'Similarity Score_FPS','Weight':'Weight_FPS','Portion Consumed':'Portion Consumed_FPS'}, inplace=True)
redFood.drop(['Source','PurEqualCon'], axis=1, inplace=True)
redFood['Food Name'] = redFood['Food Name'].str.lower()
redFood.head()

,Food Code,Food Name,Food sub-group codes,Food Group,Sales Format,Similarity Score_FPS,Weight_FPS,Portion Consumed_FPS
0,13-145,ackee,DG,Vegetables,Can,0.000000,NaN,NaN
1,13-146,agar dried,DG,Vegetables,Bag,0.276369,NaN,NaN
2,13-148,alfalfa sprouts,DG,Vegetables,Bagged produce,0.000000,NaN,NaN
3,14-896,almonds whole kernels,GA,Nuts and seeds,Bag,0.221080,NaN,NaN
4,13-149,amaranth leaves,DG,Vegetables,Bagged produce,0.467891,NaN,NaN


                                                Fruit and Vegetables

In [3]:
fv = redFood[redFood['Food Group'].str.contains('Fruit|Vegetables')]
fv['Food Group'] = fv['Food Group'].fillna('')
fv['Weight_FPS'] = fv['Weight_FPS'].fillna('')
fv['Portion Consumed_FPS'] = fv['Portion Consumed_FPS'].fillna('')
#fv.tail()

C:\Users\medekar\AppData\Local\Temp\ipykernel_25964\413163657.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fv['Food Group'] = fv['Food Group'].fillna('')
C:\Users\medekar\AppData\Local\Temp\ipykernel_25964\413163657.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fv['Weight_FPS'] = fv['Weight_FPS'].fillna('')
C:\Users\medekar\AppData\Local\Temp\ipykernel_25964\413163657.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [4]:
fv['Food Name'].value_counts().sum()

170

                                                Department of Health Data

Converted Text data to a table

In [5]:
doh = pd.read_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Data\Processed\FV_WeightData\text\DofHealth_text_FV2011.csv", index_col=0)
doh['Composite Sample Name'] = doh['Composite Sample Name'].str.lower()
doh[doh['Composite Sample Name'].str.contains('orange')]#.head()
doh['Sub-sample Number'] = doh['Sub-sample Number'].astype(str)

Concatinating All excel files into one file

In [6]:
# Set the directory path where CSV files are located
directory_path = r'C:\Users\medekar\Desktop\Product_Weight_Project\Data\Processed\FV_WeightData\tables\2011'

# Use glob to get a list of all CSV files in the directory
csv_files = glob.glob(os.path.join(directory_path, '*.csv'))

# Initialize an empty list to store DataFrames
dataframes = []

# Loop through each CSV file, read it using pandas, and append to the list
for csv_file in csv_files:
    # Read CSV file into a DataFrame
    dataframe = pd.read_csv(csv_file)
    
    # Append DataFrame to the list
    dataframes.append(dataframe)

# Concatenate all DataFrames into a single DataFrame
dataframefull = pd.concat(dataframes, ignore_index=True)

Minor Cleaning

In [7]:
dataframefull.drop(['Unnamed: 0'], axis=1, inplace=True)
dataframefull['Composite Sample Name'] = dataframefull['Composite Sample Name'].str.lower()

In [8]:
dataframefull.rename(columns={'Sub-sample Number':'Sub_sampleNumber'}, inplace = True)
dataframefull['Sub_sampleNumber'] = dataframefull['Sub_sampleNumber'].astype(str)

Matched DoH data doesn't have good weight data. I am going to go back and look at the PdF to try find better information.


Tables at the end of pdf have weight info for all samples used. Get this into text table and try matching again

                                    Updating DoH text table with Sample weights tables

In [9]:
%%time
# Define your regular expression matching function
def regex_matching(source, target):
    pattern = re.compile(f'.*{source}.*', re.IGNORECASE)
    return target if re.match(pattern, target) else ''

# Create a function to update 'Pack size' and 'Match' column based on the regex match
def update_pack_size_and_match(row):
    sub_sample_number = row['Sub-sample Number']
    
    # Find the best match in 'dataframefull' using regular expression
    best_match = dataframefull[dataframefull['Sub_sampleNumber'].apply(lambda x: regex_matching(sub_sample_number, x)) != '']
    
    if not best_match.empty:
        # Get the first match and update 'Pack size' and 'Match' column
        matched_weight = best_match.iloc[0]['Sample Weight']
        return pd.Series([matched_weight, 1], index=['Pack size', 'Match'])
    
    return pd.Series([row['Pack size'], 0], index=['Pack size', 'Match'])  # If no match, return original 'Pack size' and 'Match' as 0

# Apply the update_pack_size_and_match function to 'doh'
doh[['Pack size', 'Match']] = doh.apply(update_pack_size_and_match, axis=1)


CPU times: total: 1.83 s
Wall time: 1.82 s


                                Removing samples that didn't have the right weights attached 

In [10]:
rows_to_remove3 = [517,518,519,523,525,526,527,528,
                  530,532,533,534,535,536,537]

doh.drop(rows_to_remove3, axis=0, inplace=True)

In [11]:
rows_to_remove2 = [333,332,331,330,329,328,327,326,
                  325,324,323,322,321,320,319,318,
                  317,316,340,341,342,344,345,346,
                  347,348,350,351,352,353,354,355,
                  356,357,358,359,360,361,362,363,
                  364,365,366,367,368,369,370,371,
                  371]

doh.drop(rows_to_remove2, axis=0, inplace=True)

In [12]:
rows_to_remove1 = [659,660,661,668,669,670,671,672,673,674,675,
                   676,677,678,679,680,681,682,683,684,685,686,
                   816,815,814,810,
                   805,804,801,798]

doh.drop(rows_to_remove1, axis=0, inplace=True)

In [13]:
rows_to_remove = [717,719,722,725,782,729,732,736,738,739,740,742,747,748,753,754,756,757]

doh.drop(rows_to_remove, axis=0, inplace=True)

                                             Reducing Product list on DOH table

In [14]:
# List of food names to match
food_names = [
    'potatoes, old, raw',
    'carrots, old, raw',
    'onions, raw',
    'broccoli (calabrese), raw',
    'mushrooms, raw',
    'cabbage, white, raw',
    'cabbage, green, raw',
    'cauliflower, raw',
    'beans, green, raw',
    'tomatoes (standard), grilled',
    'lettuce, average, raw',
    'spinach, baby, raw',
    'cucumber, flesh and skin, raw',
    'peppers, red, raw',
    'peppers, yellow, raw',
    'bananas, raw, flesh only',
    'apples, eating, raw, flesh and skin',
    'apples, cooking, stewed, flesh only',
    'pears, raw, flesh and skin',
    'grapes, green, raw',
    'grapes, red, raw',
    'strawberries, raw',
    'blueberries, raw',
    'citrus fruit, soft/easy peelers, flesh only',
    'oranges, flesh only',
    'melon, yellow flesh, flesh only',
    'peas, frozen, raw',
    'sweetcorn, canned in water, drained',
    'tomatoes, canned, whole contents',
    'tomato puree',
    'baked beans in tomato sauce',
    'potato products, shaped, frozen, baked',
    'orange juice, from concentrate',
    'apple juice, clear, from concentrate'
]

# Create an empty DataFrame to store matched rows
matched_rows = []

# Create an empty DataFrame to store matched rows
matched_rows = []

# Loop through the food names and perform fuzzy matching
for food_name in food_names:
    best_similarity = 0
    best_match = None
    
    for sample_name in doh['Composite Sample Name']:
        similarity = fuzz.partial_ratio(food_name, sample_name)
        if similarity > best_similarity:
            best_similarity = similarity
            best_match = sample_name
    
    if best_match is not None:
        matched_df = doh[doh['Composite Sample Name'].str.contains(best_match, case=False, regex=True)]
        if not matched_df.empty:
            matched_row = matched_df.iloc[0].copy()
            matched_row['Similarity Score'] = best_similarity
            matched_rows.append(matched_row)

# Create a DataFrame from the matched rows
matched_food_df = pd.DataFrame(matched_rows)

C:\Users\medekar\AppData\Local\Temp\ipykernel_25964\3452392078.py:57: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  matched_df = doh[doh['Composite Sample Name'].str.contains(best_match, case=False, regex=True)]
C:\Users\medekar\AppData\Local\Temp\ipykernel_25964\3452392078.py:57: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  matched_df = doh[doh['Composite Sample Name'].str.contains(best_match, case=False, regex=True)]


In [15]:
matched_food_df.count()

Composite Sample Number          31
Composite Sample Name            32
Sub-sample Number                32
Sub-sample Name                  31
Brand                            32
Pack size                        32
Purchase Date                    31
Place of Purchase                32
Display Until Date               31
Best Before Date                 29
Batch Number                     31
Packaging                        32
Sample Condition As Purchased    30
Post Purchase Storage            27
Match                            32
Similarity Score                 32
dtype: int64

In [16]:
# Remove commas from 'Food Name' column
matched_food_df['Composite Sample Name'] = matched_food_df['Composite Sample Name'].str.replace(',', '').str.strip()

In [17]:
matched_food_df['Composite Sample Name'] = matched_food_df['Composite Sample Name'].str.replace(r'\bhomemade\b', '', case=False).str.strip()

C:\Users\medekar\AppData\Local\Temp\ipykernel_25964\1597763375.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  matched_food_df['Composite Sample Name'] = matched_food_df['Composite Sample Name'].str.replace(r'\bhomemade\b', '', case=False).str.strip()


In [18]:
matched_food_df['Composite Sample Name'] = matched_food_df['Composite Sample Name'].str.replace(r'\bbfresh\b', '', case=False).str.strip()

C:\Users\medekar\AppData\Local\Temp\ipykernel_25964\78214863.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  matched_food_df['Composite Sample Name'] = matched_food_df['Composite Sample Name'].str.replace(r'\bbfresh\b', '', case=False).str.strip()


In [19]:
matched_food_df['Composite Sample Name'] = matched_food_df['Composite Sample Name'].str.replace(r'\bboiled\b', '', case=False).str.strip()

C:\Users\medekar\AppData\Local\Temp\ipykernel_25964\2382217726.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  matched_food_df['Composite Sample Name'] = matched_food_df['Composite Sample Name'].str.replace(r'\bboiled\b', '', case=False).str.strip()


In [20]:
matched_food_df['Composite Sample Name'] = matched_food_df['Composite Sample Name'].str.replace(r'\braw\b', '', case=False).str.strip()

C:\Users\medekar\AppData\Local\Temp\ipykernel_25964\2587630719.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  matched_food_df['Composite Sample Name'] = matched_food_df['Composite Sample Name'].str.replace(r'\braw\b', '', case=False).str.strip()


In [21]:
matched_food_df['Composite Sample Name'] =matched_food_df['Composite Sample Name'].str.replace(r'\bdrained\b', '', case=False).str.strip()

C:\Users\medekar\AppData\Local\Temp\ipykernel_25964\3881204831.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  matched_food_df['Composite Sample Name'] =matched_food_df['Composite Sample Name'].str.replace(r'\bdrained\b', '', case=False).str.strip()


In [22]:
matched_food_df['Composite Sample Name'] = matched_food_df['Composite Sample Name'].str.replace(r'\bcanned\b', '', case=False).str.strip()

C:\Users\medekar\AppData\Local\Temp\ipykernel_25964\4285208305.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  matched_food_df['Composite Sample Name'] = matched_food_df['Composite Sample Name'].str.replace(r'\bcanned\b', '', case=False).str.strip()


In [23]:
matched_food_df['Composite Sample Name'] = matched_food_df['Composite Sample Name'].str.replace(r'\bretial\b', '', case=False).str.strip()

C:\Users\medekar\AppData\Local\Temp\ipykernel_25964\2315971711.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  matched_food_df['Composite Sample Name'] = matched_food_df['Composite Sample Name'].str.replace(r'\bretial\b', '', case=False).str.strip()


In [24]:
matched_food_df['Composite Sample Name'] = matched_food_df['Composite Sample Name'].str.replace(r'\bretail\b', '', case=False).str.strip()

C:\Users\medekar\AppData\Local\Temp\ipykernel_25964\2525221317.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  matched_food_df['Composite Sample Name'] = matched_food_df['Composite Sample Name'].str.replace(r'\bretail\b', '', case=False).str.strip()


In [25]:
matched_food_df['Composite Sample Name'] = matched_food_df['Composite Sample Name'].str.replace(r'\blean\b', '', case=False).str.strip()

C:\Users\medekar\AppData\Local\Temp\ipykernel_25964\1428862843.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  matched_food_df['Composite Sample Name'] = matched_food_df['Composite Sample Name'].str.replace(r'\blean\b', '', case=False).str.strip()


In [26]:
matched_food_df['Composite Sample Name'] = matched_food_df['Composite Sample Name'].str.replace(r'\bvegetable\b', '', case=False).str.strip()

C:\Users\medekar\AppData\Local\Temp\ipykernel_25964\90622374.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  matched_food_df['Composite Sample Name'] = matched_food_df['Composite Sample Name'].str.replace(r'\bvegetable\b', '', case=False).str.strip()


In [27]:
matched_food_df['Composite Sample Name'] = matched_food_df['Composite Sample Name'].str.replace(r'\bplain\b', '', case=False).str.strip()

C:\Users\medekar\AppData\Local\Temp\ipykernel_25964\1741019585.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  matched_food_df['Composite Sample Name'] = matched_food_df['Composite Sample Name'].str.replace(r'\bplain\b', '', case=False).str.strip()


In [28]:
matched_food_df['Composite Sample Name'] = matched_food_df['Composite Sample Name'].str.replace(r'\bflesh\b', '', case=False).str.strip()

C:\Users\medekar\AppData\Local\Temp\ipykernel_25964\2440311023.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  matched_food_df['Composite Sample Name'] = matched_food_df['Composite Sample Name'].str.replace(r'\bflesh\b', '', case=False).str.strip()


In [29]:
matched_food_df['Composite Sample Name'] = matched_food_df['Composite Sample Name'].str.replace(r'\band\b', '', case=False).str.strip()

C:\Users\medekar\AppData\Local\Temp\ipykernel_25964\3357240432.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  matched_food_df['Composite Sample Name'] = matched_food_df['Composite Sample Name'].str.replace(r'\band\b', '', case=False).str.strip()


In [30]:
matched_food_df['Composite Sample Name'] = matched_food_df['Composite Sample Name'].str.replace(r'\bskin\b', '', case=False).str.strip()

C:\Users\medekar\AppData\Local\Temp\ipykernel_25964\2677925965.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  matched_food_df['Composite Sample Name'] = matched_food_df['Composite Sample Name'].str.replace(r'\bskin\b', '', case=False).str.strip()


                                                    Matching M&W and DoH

In [34]:
%%time

def fuzzy_matching_score(source, target):
    return fuzz.partial_token_set_ratio(source, target)

def get_most_similar_indices(tfidf_red_food, tfidf_clean_retail):
    similarities = cosine_similarity(tfidf_red_food, tfidf_clean_retail)
    return similarities.argmax(axis=1)

def hybrid_fuzzy_matching(matched_food_df, fv, iterations=2):
    # Set initial and subsequent thresholds for matching
    initial_fuzzy_threshold = 85  # Lower the threshold for initial fuzzy matching
    subsequent_fuzzy_threshold = 99
    cosine_similarity_threshold = 0.90
    min_similarity_threshold = 0.62

    for iteration in range(iterations):
        print(f"Iteration {iteration + 1}/{iterations}")
    
          # Step 1: Fuzzy Matching (Initial: permissive, Subsequent: stricter)
        if iteration == 0:
            partial_token_set_ratio_threshold = initial_fuzzy_threshold
        else:
            partial_token_set_ratio_threshold = subsequent_fuzzy_threshold
            
        matched_food_df['Partial Token Set Ratio'] = matched_food_df['Composite Sample Name'].apply(lambda x: fuzzy_matching_score(x, fv['Food Name']))
        potential_matches = matched_food_df[matched_food_df['Partial Token Set Ratio'] >= partial_token_set_ratio_threshold]

        print('Completed 1')

        # Step 2: TF-IDF and Cosine Similarity (Text Matching) - Use combined matches as input
        vectorizer = TfidfVectorizer()
        vectorizer.fit(potential_matches['Composite Sample Name'])
        tfidf_clean_doh = vectorizer.transform(potential_matches['Composite Sample Name'])
        tfidf_red_food = vectorizer.transform(fv['Food Name'])

        # Find the most similar match for each redFood product
        max_sim_indices = get_most_similar_indices(tfidf_red_food, tfidf_clean_doh)
        max_sim_scores = cosine_similarity(tfidf_red_food, tfidf_clean_doh).max(axis=1)

        print('Completed 2')

        # Update redFood DataFrame with matched information and sources
        fv['Similarity Score DoH'] = max_sim_scores
        #fv['Weight DoH'] = potential_matches['Sample Weight'].values[max_sim_indices]
        fv['Pack Size11'] = potential_matches['Pack size'].values[max_sim_indices]
        fv['Packaging DoH11'] = potential_matches['Packaging'].values[max_sim_indices]
        fv['Purchase Date DoH11'] = potential_matches['Purchase Date'].values[max_sim_indices]


        print('Completed 3')

        # Set to NaN for rows where similarity score is below the minimum threshold
        fv.loc[fv['Similarity Score DoH'] < min_similarity_threshold, [ 'Packaging DoH11', 'Pack Size11', 'Purchase Date DoH11']] = ''
        
        # Update CleanRetail with new matches from redFood
        matched_food_df = matched_food_df.append(fv[fv['Similarity Score DoH'] >= min_similarity_threshold])


    return fv

# Call the hybrid_fuzzy_matching function and get the matched DataFrame
redFood_matched = hybrid_fuzzy_matching(matched_food_df.copy(),fv.copy())

Iteration 1/2
Completed 1
Completed 2
Completed 3
Iteration 2/2
Completed 1
Completed 2
Completed 3
CPU times: total: 484 ms
Wall time: 484 ms


<timed exec>:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


In [59]:
redFood.columns

Index(['Food Code', 'Food Name', 'Food sub-group codes', 'Food Group',
       'Sales Format', 'Similarity Score_FPS', 'Weight_FPS',
       'Portion Consumed_FPS'],
      dtype='object')

In [58]:
redFood_matched.groupby('Food Group').size()

,Food Code,Food Name,Food sub-group codes,Food Group,Sales Format,Similarity Score_FPS,Weight_FPS,Portion Consumed_FPS,Similarity Score DoH,Pack Size11,Packaging DoH11,Purchase Date DoH11
0,13-145,ackee,DG,Vegetables,Can,0.000000,,,0.000000,,,
1,13-146,agar dried,DG,Vegetables,Bag,0.276369,,,0.000000,,,
2,13-148,alfalfa sprouts,DG,Vegetables,Bagged produce,0.000000,,,0.000000,,,
4,13-149,amaranth leaves,DG,Vegetables,Bagged produce,0.467891,,,0.000000,,,
6,14-016,apples eating dried,FA,Fruit,Loose produce,0.222866,,,1.000000,128.4,Plastic,14.11.2011
7,14-031,apricots dried,FA,Fruit,Canned,0.288568,,,0.000000,,,
9,13-153,artichoke globe,DG,Vegetables,Loose produce,0.000000,,,0.000000,,,
10,13-157,asparagus,DG,Vegetables,Bunched produce,0.000000,,,0.000000,,,
11,13-161,aubergine,DG,Vegetables,Loose produce,0.000000,,,0.000000,,,
20,13-070,beans butter dried,DB,Vegetables,Can,0.859018,25g,1 portion pack,0.763698,6.0,Plastic,19.09.2011


                                                        Export Data

In [37]:
#redFood_matched.to_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Data\Processed\ReducedwithWeights\FV_MatchedFPSnDoh11.csv")

                                                       Next Notebook